## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-06-18-56-00 +0000,wsj,"Opinion | Andrew Cuomo? We’ve Been There, Done...",https://www.wsj.com/opinion/andrew-cuomo-weve-...
1,2025-08-06-18-55-00 +0000,wsj,Opinion | The Benefits of the Oil Deal With Ve...,https://www.wsj.com/opinion/the-benefits-of-th...
2,2025-08-06-18-54-42 +0000,nypost,Beloved NYC hospital worker took in her older ...,https://nypost.com/2025/08/06/us-news/beloved-...
3,2025-08-06-18-54-15 +0000,nyt,Oregon Man Accused of Killing 3 Women Is Now C...,https://www.nytimes.com/2025/08/06/us/jesse-ca...
4,2025-08-06-18-54-00 +0000,wsj,Opinion | Gerrymandering Deepens the Political...,https://www.wsj.com/opinion/gerrymandering-dee...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2300/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,58
284,new,14
43,india,13
61,will,12
47,tariffs,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
97,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...,117
290,2025-08-05-22-21-43 +0000,nypost,"Trump rips ‘no talent’ Gayle King, citing New ...",https://nypost.com/2025/08/05/media/trump-rips...,106
239,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...,106
8,2025-08-06-18-41-00 +0000,wsj,Stock Market Today: Trump Hikes India Tariffs;...,https://www.wsj.com/livecoverage/stock-market-...,106
40,2025-08-06-17-46-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...,105


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
97,117,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...
183,53,2025-08-06-10-00-00 +0000,latimes,Chabria: Newsom vows Texas will be 'neutered' ...,https://www.latimes.com/california/story/2025-...
239,48,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...
156,44,2025-08-06-11-49-45 +0000,nypost,Virgin Atlantic passenger threatens to rape fl...,https://nypost.com/2025/08/06/world-news/staff...
40,43,2025-08-06-17-46-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...
13,41,2025-08-06-18-29-53 +0000,nypost,NYC designer Martha Nolan’s business partner p...,https://nypost.com/2025/08/06/us-news/nyc-desi...
73,41,2025-08-06-16-14-13 +0000,nypost,Soldier-on-soldier shooting at Georgia’s Fort ...,https://nypost.com/2025/08/06/us-news/georgias...
61,40,2025-08-06-16-56-15 +0000,nypost,Italy approves $15.5 billion project to build ...,https://nypost.com/2025/08/06/world-news/italy...
306,36,2025-08-05-21-08-00 +0000,wsj,Opinion | China’s Z.ai and America’s Self-Defe...,https://www.wsj.com/opinion/chinas-z-ai-and-am...
53,32,2025-08-06-17-12-44 +0000,nypost,My school went phone-free — here’s what New Yo...,https://nypost.com/2025/08/06/lifestyle/my-sch...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
